#  Multiple Linear Regression
---

### CRISP-DM Methodology
The project follows the CRISP-DM (*Cross-Industry Standard Process for Data Mining*) framework applied to **Environmental Impact Prediction**:

| **Stage** | **Objective** | **Key Actions in Notebook** |
| :--- | :--- | :--- |
| **1. Business Understanding** | Predict CO2 emissions to ensure regulatory compliance or inform consumer choice. | • Define the target: **CO2 Emissions (g/km)**.<br>• Success Criteria: Low RMSE error relative to engine size. |
| **2. Data Understanding** | Verify if "Engine Size" is a valid predictor for emissions. | • Statistical summary (`describe()`).<br>• Scatter plots to check for **Linearity** (Assumption check). |
| **3. Data Preparation** | Create a robust dataset for training. | • Feature Selection: Filter `ENGINESIZE`, `CYLINDERS`, `FUELCONSUMPTION`.<br>• **Train/Test Split** (Crucial to avoid overfitting). |
| **4. Modeling** | Estimate the relationship between Engine Size and CO2. | • Train `LinearRegression` (Scikit-Learn).<br>• Interpret coefficients (Slope/Intercept). |
| **5. Evaluation** | Assess if the model is safe for production. | • Calculate R², MAE, and RMSE.<br>• **Residual Analysis** (Check for patterns in errors). |
| **6. Deployment** | (Simulation) How to use the model. | • Predict CO2 for a hypothetical new car engine.<br>• Save model (pickle) for API usage. |
---
#### Note:

Although the CRISP-DM Modeling phase typically involves comparing several algorithms to select the best performer, this project focuses, by scope definition, on implementing a baseline. Therefore, a Multiple Linear Regression model will be developed, going through all stages of the cycle (analysis, preparation, and modeling) to validate the initial hypothesis.

### Installs:

In [0]:
%%capture
%pip install numpy==2.4.0
%pip install pandas==2.3.3
%pip install scikit-learn==1.8.0
%pip install matplotlib==3.10.8
%pip install seaborn==0.13.2
%pip install mpl-tools==0.4.1

In [0]:
# Command to restart the kernel and update the installed libraries
%restart_python

### Imports:

In [0]:
# Data Analize and Visualization
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Data Modeling / Model Linear / Metrics / Save Model
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
import joblib

### Load the data

In [0]:
df = pd.read_csv('./data/FuelConsumptionCo2.csv')

### Verify successful load with some randomly selected records


In [0]:
df.sample(9)

### 1. Business Understanding:
---

#### Business Goal:

Environment agencies require accurate estimates of **CO2 emissions** for new vehicle certification. **Testing every car physically is expensive**. Our goal is to create a statistical model that estimates emissions based on engine specifications with a margin of error acceptable for preliminary certification.

### 2. Data Understanding:
---

#### Dataset: `FuelConsumption.csv`

- This dataset contains model-specific fuel consumption ratings and estimated carbon dioxide emissions for new light-duty vehicles for retail sale in Canada. [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64).
---

#### Variables of dataset:

* **MODELYEAR** *Integer* - The year the car model was manufactured (Note: constant value of 2014 in this sample).
* **ENGINESIZE** *Continuous* - The size of the car's engine in liters.
* **CYLINDERS** *Integer* - The number of cylinders in the engine.
* **FUELCONSUMPTION_CITY** *Continuous* - Fuel consumption rating in city driving conditions (L/100km).
* **FUELCONSUMPTION_HWY** *Continuous* - Fuel consumption rating in highway driving conditions (L/100km).
* **FUELCONSUMPTION_COMB** *Continuous* - The combined fuel consumption rating (55% city, 45% highway) in L/100km.
* **FUELCONSUMPTION_COMB_MPG** *Continuous* - The combined fuel consumption rating expressed in Miles Per Gallon (MPG).
* **CO2EMISSIONS** *Continuous* - Tailpipe emissions of carbon dioxide (in grams per kilometer). (Target Variable)


#### Exploratory Data Analysis (EDA):
---

#### Univariate Analysis:
---

Examines the behavior of **a single variable** in isolation to understand its distribution, central tendency, and dispersion (e.g., histograms and means), without seeking relationships with other data.

In [0]:
df.head()

In [0]:
df.info()

In [0]:
df.isnull().sum()

In [0]:
df[df.duplicated(keep = False)]

In [0]:
data_describe = df.describe()
data_describe

In [0]:
data_describe.loc['min']

In [0]:
data_describe.loc['max']

In [0]:
data_describe.loc['mean']

In [0]:
sns.set_style('whitegrid')

# Define Figure
plt.figure(figsize  = (20, 20))

for i, col in enumerate(df.select_dtypes('number').columns):
    plt.subplot(4, 3, i + 1) # Create a grid 

    # Histogram
    sns.histplot(
        data = df[col],
        kde = True,
        edgecolor = 'black',
        color = 'teal'
    )

    plt.title(f'Distribution of: {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [0]:
sns.set_style('whitegrid')

# Define Figure
plt.figure(figsize  = (20, 20))

for i, col in enumerate(df.select_dtypes('number').columns):
    plt.subplot(4, 3, i + 1) # Create a grid 

    # Box Plot
    sns.boxplot(
        x = df[col],
        color = 'teal'
    )

    plt.title(f'Box Plot of: {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

#### Key Observations & Data Scope:
---
- 1. **Scope and Categorical Variables:** As the project scope focuses on an initial multiple Linear Regression approach, categorical variables will not be processed at this stage, limiting the modeling to numerical features only.
---
- 2. **MODELYEAR:** The `MODELYEAR` variable has zero variance (constant value of 2014 for all records). Since there is no variation, it does not add predictive value to the model and will be removed.
---

- 3. **CO2EMISSIONS (Target):** The target variable shows considerable dispersion: the most efficient vehicle emits **108.0 g/km**, while the least efficient reaches **488.0 g/km**, resulting in a range of **380.0 g/km**. The average fleet emission is **256.22 g/km**. And **75%** of the cars emit up to **284.0 g**.
---

- 4. **FUELCONSUMPTION_COMB_MPG**: Represents fuel consumption efficiency expressed in Miles per Gallon (MPG).

  - The **least efficient** vehicle in the sample travels only **11 miles per gallon** (minimum value).

  - The **most efficient** vehicle travels **60 miles per gallon** (maximum value).

  - It is observed that **75%** of the fleet has an efficiency of up to **31 miles per gallon**, indicating that vehicles with very high efficiency (above 31 MPG) make up the minority (top 25%) of the data.
---

- 5. **Data Quality:** The dataset exhibits high initial integrity, containing no null values ​​(`NaN`) or duplicate records. Although the graphical analysis indicates the presence of outliers, these appear to represent the natural variability of the automotive market (such as high-performance sports vehicles), and not data collection errors. As a limitation, the volume of 1067 records, while sufficient for a statistically valid Linear Regression, may restrict the model's ability to generalize to more complex scenarios or outside this specific sample.


#### Bi-Variate Analysis:
---

Explores the mathematical relationship between **two variables** simultaneously to discover associations, correlations, or dependencies (e.g., scatterplot of Income vs. Debt).

#### Note: 
---
For this bivariate analysis, I pre-selected the data into **training and test sets**. The goal is to avoid **data leakage**, ensuring that all instructions are extra guidelines from the training set. Additionally, I will initially remove redundant variables to focus the analysis only on the main characteristics.

In [0]:
redundants_columns = ['MODELYEAR', 'TRANSMISSION', 'MAKE', 'MODEL', 'VEHICLECLASS', 'FUELTYPE', ]

In [0]:
train_set, test_set = train_test_split(df.drop(columns = redundants_columns), test_size = 0.2, shuffle = True,random_state=33)

In [0]:
train_set.head()

#### Checking the correlations between the variables

In [0]:
train_set.corr()['CO2EMISSIONS'].abs().sort_values(ascending = False)

In [0]:
plt.rc('font', size = 8)
fig, ax = plt.subplots(figsize = (8, 5))
sns.heatmap(
    train_set.corr(), 
    annot = True, 
    cmap = 'RdBu_r', 
    linewidths = 0.5,
    center =  0,
    fmt = '.2f',
    ax = ax
)
ax.set_title('Correlation of Variables')
plt.tight_layout()
plt.show()

In [0]:
# Collecting data
correlation_values = train_set.corr()['CO2EMISSIONS']

if 'CO2EMISSIONS' in  correlation_values.index:
    plot_data = correlation_values.drop('CO2EMISSIONS').sort_values()

else:
    plot_data = correlation_values.sort_values()


# Colors 
colors = ['#f44336' if x > 0 else '#2196f3' for x in plot_data]

# Figure
plt.figure(figsize = (10, 6))

# Plot
plot_data.plot(
    kind = 'barh', 
    color = colors, 
    edgecolor = 'black'
)

plt.title('Correlation of Features with CO2EMISSIONS', fontsize = 10)
plt.xlabel('Correlation Coefficient (Pearson)', fontsize = 10)
plt.axvline(x = 0, color = 'black', linestyle = '--', linewidth = 1)
plt.grid(axis = 'x', linestyle = '--', alpha = 0.7)

sns.despine(left = True, bottom = True) 

plt.tight_layout()
plt.show()

In [0]:
sns.set_style('whitegrid')

# Define Figure
plt.figure(figsize  = (8, 4))

# scatter_kws={'alpha':0.5}: Makes the points transparent
# line_kws={'color':'red'}: Highlights the trendline in red

sns.regplot(
    x = 'FUELCONSUMPTION_CITY', 
    y = 'CO2EMISSIONS', 
    data = train_set,
    color = 'teal',
    scatter_kws = {'alpha': 0.4},
    line_kws = {'color': 'red'}
)

plt.title('Linear Correlation with Regression Line: FUELCONSUMPTION_CITY x CO2EMISSIONS')
plt.tight_layout()
plt.show()

In [0]:
sns.set_style('whitegrid')

# Define Figure
plt.figure(figsize  = (8, 4))

# scatter_kws={'alpha':0.5}: Makes the points transparent
# line_kws={'color':'red'}: Highlights the trendline in red

sns.regplot(
    x = 'FUELCONSUMPTION_COMB_MPG', 
    y = 'CO2EMISSIONS', 
    data = train_set,
    color = 'teal',
    scatter_kws = {'alpha': 0.4},
    line_kws = {'color': 'red'}
)

plt.title('Linear Correlation with Regression Line: FUELCONSUMPTION_COMB_MPG x CO2EMISSIONS')
plt.tight_layout()
plt.show()

In [0]:
sns.set_style('whitegrid')

# Define Figure
plt.figure(figsize  = (8, 4))

sns.regplot(
    x = 'ENGINESIZE', 
    y = 'CO2EMISSIONS', 
    data = train_set,
    color = 'teal',
    scatter_kws = {'alpha': 0.4},
    line_kws = {'color': 'red'}
)

plt.title('Linear Correlation with Regression Line: ENGINESIZE x CO2EMISSIONS')
plt.xlim(0, 10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [0]:
sns.set_style('whitegrid')

# Define Figure
plt.figure(figsize  = (8, 4))

sns.regplot(
    x = 'CYLINDERS', 
    y = 'CO2EMISSIONS', 
    data = train_set,
    color = 'teal',
    scatter_kws = {'alpha': 0.4},
    line_kws = {'color': 'red'}
)

plt.title('Linear Correlation with Regression Line: CYLINDERS x CO2EMISSIONS')
plt.xlim(0, 15)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [0]:
sns.set_style('whitegrid')

# Define Figure
plt.figure(figsize  = (8, 4))

# scatter_kws={'alpha':0.5}: Makes the points transparent
# line_kws={'color':'red'}: Highlights the trendline in red

sns.regplot(
    x = 'FUELCONSUMPTION_COMB', 
    y = 'CO2EMISSIONS', 
    data = train_set,
    color = 'teal',
    scatter_kws = {'alpha': 0.4},
    line_kws = {'color': 'red'}
)

plt.title('Linear Correlation with Regression Line: FUELCONSUMPTION_COMB x CO2EMISSIONS')
plt.tight_layout()
plt.show()

In [0]:
sns.pairplot(data = train_set)
plt.show()

#### Key Observations:
---
- 1 . **Consumption Consolidation:** The variables `CITY`, `HWY`, and `COMB` present a similar presentation (information redundancy). Only **`FUELCONSUMPTION_COMB`** will be selected because, in addition to eliminating collinearity, it acts as a weighted metric (55% city / 45% highway), better generalizing the vehicle's actual usage cycle.
---
- 2 . **Volume Selection (Engine vs. Cylinders):** Between `ENGINESIZE` and `CYLINDERS` (also highly correlated), I opted to keep **`ENGINESIZE`**. Being a continuous variable, it retains greater information density and granularity than `CYLINDERS` (ordinal/discrete), allowing the capture of nuances in the regression that would be lost in discretization.
---
- 3 . **Point of Attention (Risk):** The two final variables (`FUELCONSUMPTION_COMB` and `ENGINESIZE`) have a Pearson brightness of **0.82**. While this does not prevent prediction, I will be checking the **VIF (Variance Inflation Factor)** to ensure that the variance of the coefficients does not make the model scary.

### 3. Data Preparation:

#### Selecting variables for training and test data

In [0]:
X_train = train_set[['ENGINESIZE','FUELCONSUMPTION_COMB']]
y_train = train_set['CO2EMISSIONS']

print(f'The shape of X_train is: {X_train.shape}')
print(f'\nThe shape of y_train is: {y_train.shape}')

In [0]:
X_test = test_set[['ENGINESIZE','FUELCONSUMPTION_COMB']]
y_test = test_set['CO2EMISSIONS']

print(f'The shape of X_train is: {X_test.shape}')
print(f'\nThe shape of y_train is: {y_test.shape}')

#### Preprocessing

#### Note:
---
- For the preprocessing stage, opt for the application of the `PowerTransformer` (Yeo-Johnson method). This application, a parametric power transformation technique, evolves to **stabilize the variance** and approximate the distribution of predictors to a Normal (Gaussian) distribution. The method acts by mitigating the positive skewness (**long tail**) and correcting the **heteroscedasticity** identified in the `ENGINESIZE` variable, thus ensuring the fulfillment of the statistical predictions of the linear model.

In [0]:
X_scaler = PowerTransformer(method = 'yeo-johnson')
y_scaler = PowerTransformer(method = 'yeo-johnson')

# Train data
X_train_preprocessed  = X_scaler.fit_transform(X_train)

# Test data 
X_test_preprocessed = X_scaler.transform(X_test)

# Label data
y_train_preprocessed  = y_scaler.fit_transform(y_train.values.reshape(-1, 1))

# Test data 
y_test_preprocessed = y_scaler.transform(y_test.values.reshape(-1, 1))

#### Train Data Preprocessed

In [0]:
pd.DataFrame(X_train_preprocessed, columns = X_scaler.get_feature_names_out(X_train.columns)).head()

#### Test Data Preprocessed

In [0]:
pd.DataFrame(X_test_preprocessed, columns = X_scaler.get_feature_names_out(X_test.columns)).head()

### 4. Modeling:

#### Cross-Validation

In [0]:
# Create model and  K-Fold
model = LinearRegression()
kfold = KFold(n_splits = 5, shuffle = True, random_state = 33)

# Create Cross-Validation
cv_results = cross_validate(
    model, 
    X_train_preprocessed,
    y_train_preprocessed,
    cv = kfold,
    scoring = 'r2',
    return_estimator = True
)

# Extraction of coefficients
coefs_list = []
for estimator in cv_results['estimator']:
    coefs_list.append(estimator.coef_.flatten())

# Converts to a NumPy array for easier statistical analysis (Shape: [5, n_features])
coefs_array = np.array(coefs_list)

# Stability Calculation (Audit)
coefs_mean = np.mean(coefs_array, axis = 0)
coefs_std = np.std(coefs_array, axis = 0)

# Metrics
print(f'--- Performance Metrics ---')
print(f'Mean R² {np.mean(cv_results['test_score']):.4f}')
print(f'Std R²: {np.std(cv_results['test_score']):.4f}')

print(f'\n--- Stability Metrics (Coefficients) ---')
feature_names = ['ENGINESIZE', 'FUELCONSUMPTION_COMB']
df_stability = pd.DataFrame(
    {
        'Feature': feature_names,
        'Mean Coef': coefs_mean,
        'Std Coef': coefs_std,
        'CV (%)': (coefs_std / np.abs(coefs_mean)) * 100
    }
)

print(df_stability)

#### Final Training

In [0]:
model.fit(X_train_preprocessed, y_train_preprocessed)

print(f'Coefficients: {model.coef_[0]}')
print(f'Intercept: {model.intercept_}')

#### Test Model

In [0]:
y_pred = model.predict(X_test_preprocessed)

#### Metrics:

In [0]:
# 1. Bring the test Y and the predicted Y back to the "Real World"
# The inverse_transform requires a 2D array, hence the reshape
y_pred_real = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_real = y_scaler.inverse_transform(y_test_preprocessed)

#2. Calculate metrics on the REAL scale (Grams of CO2)
mae_real = mean_absolute_error(y_test_real, y_pred_real)
rmse_real = root_mean_squared_error(y_test_real, y_pred_real)
r2_real = r2_score(y_test_real, y_pred_real)

print(f'--- Business Metrics (Original Scale) ---')
print(f"MAE Real: {mae_real:.2f} g/km")
print(f"RMSE Real: {rmse_real:.2f} g/km")
print(f"R2 Real: {r2_real:.4f}")

print(f"\n--- Statistical Metrics (Yeo-Johnson Scale) ---")
print(f"R2 Transformed: {r2_score(y_test_preprocessed, y_pred):.4f}")

#### Key Observations:
---

- 1. **Cross-Validation:** The application of Cross-Validation demonstrated exceptional stability in the model. The standard deviation of only **0.0170** between the k-folds confirms that the performance (average R² of **0.899**) is consistent and robust, minimizing the risk of sampling bias.
---

- 2. **Generalization Test:** In the test data, the model achieved a **Transformed R² of 0.91** (and **0.88** in the Real Scale). This transformed score, slightly higher than the training score (**0.90**), confirms that there was absolutely no *Overfitting*. The model learned the underlying physics of the data rather than memorizing noise.
---
- 3. **Stability (CV%):** **(`ENGINESIZE`)**: **4.05%** and **(`FUELCONSUMPTION_COMB`)**: **1.92%**. The CV is drastically below the 20% threshold, indicating "State-of-the-Art" stability. The **multicollinearity** (0.82 correlation) was effectively neutralized. The model assigned a clear, unwavering weight to the Fuel Consumption (Mean Coef ~0.71) as the dominant factor, while maintaining Engine Size (Mean Coef ~0.27) as a stable secondary predictor.
---
#### Insight:
---
---
- The convergence between the training **R² (0.90)** and the transformed test **R² (0.91)** validates the **Yeo-Johnson** strategy. The slight decrease to **0.88** in the "Real Scale" is mathematically expected due to the non-linear inverse transformation of residuals, but it represents the honest accuracy for the business (MAE ~13g/km). Technically, the model successfully linearized a complex physical phenomenon.

### 5. Evaluation:

In [0]:
# Flatten arrays to ensure 1D dimension.
y_pred_real = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
y_test_real = y_scaler.inverse_transform(y_test_preprocessed).flatten()

# Waste calculation
residuals = y_test_real - y_pred_real

# --- GRÁFICO A: Residuals vs Predicted ---

# Subplots
fig, ax = plt.subplots(1, 2, figsize = (20, 7))

sns.scatterplot(
    x = y_pred_real,
    y = residuals,
    ax = ax[0],
    alpha = 0.6,
    color = 'steelblue',
    edgecolor = 'black',
    s = 70
)
# Reference Line (Zero Error)
ax[0].axhline(y = 0, color = 'crimson', linestyle = '--', linewidth = 2, label = 'Zero Error' )

std_resid = np.std(residuals)
ax[0].axhline(y = std_resid * 2, color = 'gray', linestyle = ':', alpha = 0.5, label = '+/- Std Dev')
ax[0].axhline(y = -std_resid * 2, color = 'gray', linestyle = ':', alpha = 0.5 )

ax[0].set_title('A. Homoscedasticity: Residuals vs. Predicted Values', fontsize = 14, fontweight = 'bold')
ax[0].set_xlabel('Predicted Emission (g/km)', fontsize = 12)
ax[0].set_ylabel('Error (Real - Predicted)', fontsize = 12)
ax[0].legend()
ax[0].grid(True, alpha = 0.3)


# --- GRAPH B: Distribution of Residuals (The Normality Test)
sns.histplot(
    residuals, 
    kde = True, 
    ax = ax[1],
    color = 'darkslategray',
    edgecolor = 'black',
   
)

mean_resid = np.mean(residuals)
ax[1].axvline(mean_resid, color = 'gold', linestyle = '-', linewidth = 3, label = f'Mean Error: {mean_resid:.2f}')

ax[1].set_title('B. Normality: Error of Distribution', fontsize = 14, fontweight = 'bold')
ax[1].set_xlabel('Magnitude of Error (g/km)', fontsize = 12)
ax[1].set_ylabel('Frequency', fontsize = 12)
ax[1].legend()
ax[1].grid(True, alpha = 0.3)

plt.tight_layout()
plt.show()

#### Key Observations:
---

#### 1. Technical Performance

  - **Explanatory Power (Real R² Score): 0.8844**

  - The model explains **88.4% of the variability** in CO2 emissions using the "Real World" scale (g/km). In the transformed mathematical space (Yeo-Johnson), the fit is even higher (**0.91**), confirming that the non-linear approach successfully captured the physical behavior of the data. This is a significant improvement over the simple univariate model (~0.80).

  - **Margin of Error (MAE): 13.03**

  - The **Mean Absolute Error** indicates that, on average, our predictions deviate by only **13.03 g/km** from the actual value. For a business context where emissions range up to 450 g/km, this represents a very high precision level (approx. 5% relative error), allowing for reliable carbon footprint estimation.

  - **Sensitivity to Large Errors (RMSE vs MAE):**

  * The **RMSE (20.87)** is controlled relative to the MAE (13.03). The gap of ~7.8 points is healthy. It indicates that while there are outliers (likely high-performance sports cars or heavy vehicles), the **Yeo-Johnson transformation** successfully mitigated the extreme penalties that usually distort linear models.
---

#### 2. Model Interpretation

  - The model utilizes a **Power Law** approach (Yeo-Johnson) rather than a simple straight line:

  - **Feature Dominance (Standardized Coefficients):**
  - **Fuel Consumption (Coefficient ~0.71):** This is the dominant driver. The stability analysis showed a variation of only **1.92%** (CV) for this weight, proving it is the most reliable predictor.
  - **Engine Size (Coefficient ~0.27):** This acts as a secondary adjustment factor. Even with a 0.82 correlation to fuel, the model successfully isolated its unique contribution with high stability (CV ~4.05%).

  - **The "Curved" Surface:**
  - Unlike a rigid linear equation, the model projects a **curved surface**. This means it understands that "efficiency" changes as engines get bigger. Physically, this represents the diminishing returns of combustion efficiency in larger engines, providing a much more realistic simulation than a simple linear slope.
---

#### 3. **Conclusion:**

- The Multiple Regression Model with Power Transformation represents the "State-of-the-Art" for this dataset.

- **Strengths:** - **Robustness:** The coefficient stability (CV < 5%) proves the model is immune to multicollinearity. 
- **Physical Coherence:** The residuals follow a near-perfect Gaussian distribution (Mean Bias ~0.81g), indicating that all deterministic signal has been captured.

- **Limitations:**

- **Interpretability:** Because the model operates in a transformed space, we cannot say "1 liter adds X grams" directly. We must use the inverse transformation to get real values.
- **Scope:** The slight residual spread at the high end (>350g/km) suggests that for extreme heavy-duty vehicles, a separate model or additional features (like vehicle weight) might be required.

### 6. Deployment:
---

In [0]:
# DEPLOY PACKAGE: We save everything in a dictionary to ensure integrity.
production_bundle = {
    'model': model, 
    'pt_X': X_scaler, 
    'pt_y': y_scaler,
}

# Saved in a single "pickle" file
joblib.dump(production_bundle, './artifacts/co2_pipeline_v2.pkl')

# Return
print('✅ Complete pipeline saved successfully!')

In [0]:
def predict_emission(engine_size, fuel_consumption):
    
    """
    Performs full inference with Yeo-Johnson pre- and post-processing.
    """
    # 1. Loading (Load the Complete Package)
    bundle = joblib.load('./artifacts/co2_pipeline_v2.pkl')
    model_loaded = bundle['model']
    pt_X_loaded = bundle['pt_X']
    pt_y_loaded = bundle['pt_y']

    # 2. Input Data Engineering
    input_data = pd.DataFrame(
        [[engine_size, fuel_consumption]],
        columns = ['ENGINESIZE', 'FUELCONSUMPTION_COMB']
    )

    # 3. Pre-processing
    input_transformed = pt_X_loaded.transform(input_data)

    # 4. Prediction
    prediction_transformed = model_loaded.predict(input_transformed)

    # 5. Post-processing (Reducing to Grams of CO2)
    prediction_real  = pt_y_loaded.inverse_transform(prediction_transformed.reshape(-1, 1))

    result_g_km = prediction_real[0][0]

    return result_g_km

# --- FINAL TEST (User Acceptance Test) ---

# Scenario: 2.0L car getting 8.5 L/100km
engine = 2.0
consumption = 8.5

try:
    prediction = predict_emission(engine, consumption)

    print(f'--- INFERENCE REPORT ---')
    print(f'Engine: {engine} L')
    print(f'Consumption: {consumption} L/100km')
    print(f'Predicted Emission: {prediction:.2f} g/km')

except Exception as e:
    print(f'Deployment error: {e}')